In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Libraries
import pandas as pd
import os

from src.data import prep_data as prep
from src.models import model_schemata as schema
from src.models import build_model as build

In [3]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [8]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/sub_network_n/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/sub_network_n/p_fc.csv')

In [12]:
e_ksea_ad = prep.expand_ad_cpd(data['e_ksea'], 'value')
p_fc_ad = prep.expand_ad_cpd(data['p_fc'], 'value')
p_fc_ad

,phosphosite,sample,value,prob,p_dec,p_inc,p_base
0,ABI1(Y213),AC220,dec,0.317612,0.317612,0.001000,0.681388
1,ABI1(Y213),AC220,inc,0.317612,0.001000,0.317612,0.681388
2,ABI1(Y213),AT13148,dec,0.046184,0.046184,0.001000,0.952816
3,ABI1(Y213),AT13148,inc,0.046184,0.001000,0.046184,0.952816
4,ABI1(Y213),AZ20,dec,0.012765,0.012765,0.001000,0.986235
...,...,...,...,...,...,...,...
2313,SRC(S75),U73122,inc,0.753525,0.001000,0.753525,0.245475
2314,SRC(S75),Ulixertinib,dec,0.564628,0.564628,0.001000,0.434372
2315,SRC(S75),Ulixertinib,inc,0.564628,0.001000,0.564628,0.434372
2316,SRC(S75),Vemurafenib,dec,0.515025,0.515025,0.001000,0.483975


In [13]:
# Mapping data to Problog predicates
# make sure to delete 'base' from schemata and AnnotatedDisjunctionClauseTemplate
predicates = {}
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(e_ksea_ad, 'enzyme', 'sample', 'value')
predicates['e_ksea'].add_ad_probabilities(e_ksea_ad, p_dec_col='p_dec', p_base_col='p_base', p_inc_col='p_inc')
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(p_fc_ad, 'phosphosite', 'sample', 'value')
predicates['p_fc'].add_ad_probabilities(p_fc_ad, p_dec_col='p_dec', p_base_col='p_base', p_inc_col='p_inc')

In [15]:
model_lfi = 'models/ebdt_data/sub_network/n_model/n_model_lfi_exp1.pl'
for predicate in predicates: 
    ad_generator = build.ProblogStatementGenerator(predicates[predicate])
    clauses = ad_generator.generate_ad_clauses(type='boolean')
    clauses.extend(ad_generator.generate_ad_clauses(type='domain'))
    build.insert_statements(model=model_lfi, statements=clauses, location='%% p1::{}'.format(predicate)) # insert into Problog file